In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from lyricsgenius import Genius

In [2]:
GENIUS_TOKEN = "s2suIFNrg8eTLm-6UGm5SHyXWsDcdE0517ahKBd-mysrN-klE1Q5hRkAQYugYpbS"
genius = Genius(
    GENIUS_TOKEN,
    verbose=False,
    remove_section_headers=True
)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
}


In [3]:
def scrape_lyrics(song_url):
    try:
        r = requests.get(song_url, headers=HEADERS, timeout=10)
        r.raise_for_status()
        
        soup = BeautifulSoup(r.text, "html.parser")

        # New Genius layout (2023+)
        containers = soup.select("div[data-lyrics-container='true']")
        if not containers:
            return None

        lyrics = "\n".join(c.get_text(separator="\n") for c in containers)
        return lyrics.strip()
    
    except Exception:
        return None



In [4]:
# Load dataset
df = pd.read_csv("dataset/billboard200.csv")

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter for year 2025
df_2025 = df[df['Date'].dt.year == 2025]

# Get unique songs (Song, Artist)
songs_to_collect = df_2025[['Song', 'Artist']].drop_duplicates().values.tolist()

print(f"Found {len(songs_to_collect)} songs from 2025")


Found 637 songs from 2025


/var/folders/nk/t4_wkj7d76jbr43cg2yhqnb00000gn/T/ipykernel_10784/1313961295.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset/billboard200.csv")


In [5]:
rows = []

for title, artist in songs_to_collect:
    print(f"🎵 Processing: {title} – {artist}")
    
    try:
        results = genius.search_songs(f"{title} {artist}")
        hit = results["hits"][0]["result"]
        song_id = hit["id"]
        
        song = genius.song(song_id)
        song_url = song["song"]["url"]
        
        lyrics = scrape_lyrics(song_url)
        
        rows.append({
            "title": song["song"]["title"],
            "artist": song["song"]["primary_artist"]["name"],
            "url": song_url,
            "lyrics": lyrics,
            "has_lyrics": lyrics is not None
        })
        
        print("   ✅ done")
        time.sleep(3)  # important: be polite
    
    except Exception as e:
        print("   ❌ failed")



🎵 Processing: SOS – SZA
   ✅ done
🎵 Processing: Christmas – Michael Buble
   ✅ done
🎵 Processing: Ultimate Christmas – Bing Crosby
   ✅ done
🎵 Processing: The Christmas Song – Nat King Cole
   ✅ done
🎵 Processing: GNX – Kendrick Lamar
   ✅ done
🎵 Processing: Merry Christmas – Mariah Carey
   ✅ done
🎵 Processing: The Tortured Poets Department – Taylor Swift
   ✅ done
🎵 Processing: Wicked: The Soundtrack – Cynthia Erivo| Ariana Grande & Various Artists
   ❌ failed
🎵 Processing: A Christmas Gift For You From Phil Spector – Various Artists
   ✅ done
🎵 Processing: Ultimate Christmas – Frank Sinatra
   ✅ done
🎵 Processing: A Charlie Brown Christmas (Soundtrack) – Vince Guaraldi Trio
   ✅ done
🎵 Processing: Short n' Sweet – Sabrina Carpenter
   ✅ done
🎵 Processing: Hop – Stray Kids
   ✅ done
🎵 Processing: Hit Me Hard And Soft – Billie Eilish
   ✅ done
🎵 Processing: Rockin' Around The Christmas Tree: The Decca Christmas Recordings – Brenda Lee
   ❌ failed
🎵 Processing: The Rise And Fall Of A M

In [6]:
df = pd.DataFrame(rows)
df.to_csv("output/songs_with_lyrics.csv", index=False)


